In [65]:
import numpy as np
import pandas as pd
import en_core_web_sm
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
from sklearn.preprocessing import OneHotEncoder
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
from pydotplus import graph_from_dot_data
import pandas
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [66]:
# #imports
# import pandas as pd
# import numpy as np
# from gensim.models import Word2Vec
from nltk import word_tokenize
# import spacy 
# import en_core_web_sm
# from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.lang.en import English
# import pandas as pd
# from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# from sklearn.pipeline import Pipeline

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()


In [67]:
df = pd.read_csv('sentence_features.csv')

In [68]:
df.head()

,article_id,propaganda,propaganda_type,text,prop_txt_snippet,sent_#,sentiment_score,abs_sent_score,punct_count,word_count,%adj,%verb,%adv,%noun,avg_word_length
0,701225819,non-propaganda,NaN,South Florida Muslim Leader Sofian Zakkout’s D...,NaN,1,0.0000,0.0000,0,9,0.000000,0.000000,0.000000,0.000000,5.444444
1,701225819,propaganda,"Name_Calling,Labeling","David Duke, the white supremacist icon and for...",Grand Wizard of the Ku Klux Klan,2,0.5423,0.5423,4,26,0.020548,0.006849,0.013699,0.006849,4.423077
2,701225819,propaganda,Loaded_Language,"However, one individual who represents the Mus...",enamored,3,0.3612,0.3612,4,27,0.017241,0.017241,0.005747,0.022989,5.000000
3,701225819,non-propaganda,NaN,"Last month, once again, Zakkout chose to showc...",NaN,4,0.0000,0.0000,5,22,0.021127,0.021127,0.014085,0.035211,5.045455
4,701225819,non-propaganda,NaN,The postings can be rivaled only by Zakkout’s ...,NaN,5,0.0000,0.0000,1,11,0.014493,0.043478,0.014493,0.028986,4.636364


In [69]:
model_1 = df.drop(['article_id','propaganda_type','prop_txt_snippet','sent_#'], axis=1)

In [70]:
model_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15220 entries, 0 to 15219
Data columns (total 11 columns):
propaganda         15220 non-null object
text               15220 non-null object
sentiment_score    15220 non-null float64
abs_sent_score     15220 non-null float64
punct_count        15220 non-null int64
word_count         15220 non-null int64
%adj               15220 non-null float64
%verb              15220 non-null float64
%adv               15220 non-null float64
%noun              15220 non-null float64
avg_word_length    15172 non-null float64
dtypes: float64(7), int64(2), object(2)
memory usage: 1.3+ MB


In [71]:
model_1 = model_1[model_1['avg_word_length'].isna()==False]

# Tokenizer

In [72]:
#IMPORTS
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string

punctuation = string.punctuation
punctuation = punctuation+"..."+"--"+"“"+"”"+"``"+"''"+"’"+"–"+"—"+"‘"
lemmatizer = WordNetLemmatizer()

In [73]:
contr_dict={"I’m": "I am",
            "won’t": "will not",
            "’s" : "", 
            "’ll":"will",
            "’ve ":"have ",
            "n’t":" not",
            "’re": "are",
            "’d": "would",
            "y’all": "all of you",
            "I'm": "I am",
            "won't": "will not",
            "'s" : "", 
            "'ll":"will",
            "'ve ":"have ",
            "n't":"not",
            "'re": "are",
            "'d": "would",
            "y'all": "all of you"}
contr_dict.keys()


dict_keys(['I’m', 'won’t', '’s', '’ll', '’ve ', 'n’t', '’re', '’d', 'y’all', "I'm", "won't", "'s", "'ll", "'ve ", "n't", "'re", "'d", "y'all"])

In [74]:
def replace_contractions(sentence, contr_dict=contr_dict):
    for contr in contr_dict.keys():
        if contr in sentence:
            sentence = sentence.replace(contr,contr_dict[contr])
    return sentence

In [75]:
# function to convert nltk tag to wordnet tag
# this is important because having the POS tag improves lemmatization
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [76]:
#tokenization and lemmatization function
def tokenize_sentence(sentence):
    #remove contractions
    sentence = replace_contractions(sentence, contr_dict=contr_dict)
    
    #tokenize the sentence
    mytokens = nltk.word_tokenize(sentence)
    
    #tag tokens with part of speech
    nltk_tagged = nltk.pos_tag(mytokens)
    
    # remove punctuation
    nltk_tagged = [ word for word in nltk_tagged if word[0] not in punctuation ]
    
    # strip all tokens and make lowercase 
    nltk_tagged = [ (word[0].lower().strip(),word[1]) for word in nltk_tagged ]
    
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    
    lemmatized_tokens = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_tokens.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_tokens.append(lemmatizer.lemmatize(word, tag))
            
        lemmatized_tokens = [word for word in lemmatized_tokens if word not in stop_words]
    return lemmatized_tokens


In [245]:
#testing tokenizer
tokenize_sentence(text_df['text'].loc[0])

NameError: name 'text_df' is not defined

# Train-Test-Split

In [78]:
from sklearn.model_selection import train_test_split
y= model_1['propaganda']
X = model_1.drop('propaganda',axis=1)


In [79]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15172 entries, 0 to 15219
Data columns (total 10 columns):
text               15172 non-null object
sentiment_score    15172 non-null float64
abs_sent_score     15172 non-null float64
punct_count        15172 non-null int64
word_count         15172 non-null int64
%adj               15172 non-null float64
%verb              15172 non-null float64
%adv               15172 non-null float64
%noun              15172 non-null float64
avg_word_length    15172 non-null float64
dtypes: float64(7), int64(2), object(1)
memory usage: 1.9+ MB


In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Apply Transformations to Features:
* TFIDF for text data
* Scaling for numerical features like Word Count

In [ ]:
# corpus = [
#     'This is the first document.',
#     'This document is the second document.',
#     'And this is the third one.',
#     'Is this the first document?',
# ]
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus)
# print(vectorizer.get_feature_names())

# print(X.shape)

# Methods

In [98]:
tfidf_vector = TfidfVectorizer(tokenizer = tokenize_sentence, 
                               min_df=10, max_df=0.7)

In [99]:
scaler = sklearn.preprocessing.StandardScaler()

In [100]:
from sklearn_pandas import DataFrameMapper, cross_val_score

In [101]:
training_text = X_train['text']

In [127]:
non_text_train = X_train.drop('text',axis=1)

In [140]:
scaler = StandardScaler()
scaled_train = scaler.fit_transform(non_text_train)

In [130]:
scaled_train = pd.DataFrame(scaled_train)
scaled_train.columns = list(non_text_train.columns)

In [131]:
scaled_train.head()

,sentiment_score,abs_sent_score,punct_count,word_count,%adj,%verb,%adv,%noun,avg_word_length
0,0.378942,-0.584117,0.107380,1.787735,0.125383,0.155285,-0.298495,0.849950,-0.302474
1,0.069278,-1.026921,-0.712079,-0.409388,0.314773,0.438876,-0.540039,0.792049,-0.648888
2,0.069278,-1.026921,0.517110,-0.182099,-0.861440,0.552504,-0.080325,-0.273210,0.703670
3,0.069278,-1.026921,1.336569,0.045189,-0.277755,-0.448369,-0.540039,0.398204,0.460798
4,0.069278,-1.026921,-0.712079,-1.091253,-0.861440,0.830264,-0.540039,0.370299,-0.448944


In [102]:
training_text 

13046    As I wrote in my first analysis piece, the doc...
13984    Could you give us a feel for the kind of perso...
12566    The New Yorker article, written before Ms. For...
9400     [I]f one arrives at the recognition that, in a...
14843                          The body has been cremated!
4899     When someone is shooting form a horizontal lin...
731      This because Trump retweeted three videos show...
11010    Bolton says it will be accomplished in the nex...
12540    But Campos has been a mysterious subject since...
6296     Ostensibly, the next six months are needed to ...
10240    Such circular “reasoning” would be amusing if ...
1749     Completing this poll grants you access to Free...
3862     They can write all big and bad but when it com...
9814     do that which is good, and thou shalt have pra...
14852    “The shooter’s body was cremated Dec. 21," sai...
2714     Lew said he feels vindicated to some extent, b...
9515     Ever since being soundly beaten in Vietnam and.

In [108]:
tfidf_text_train = tfidf_vector.fit_transform(training_text)

In [111]:
tfidf_text_train_df = pd.DataFrame(tfidf_text_train.toarray())

In [122]:
tfidf_text_train_df.columns = list(tfidf_vector.get_feature_names())

In [120]:
tfidf_text_train_df.shape

(10165, 1958)

In [125]:
tfidf_text_train_df.head()

,1,10,11,12,13,14,15,16,17,18,...,yore,york,youare,young,youth,zakkout,zarate,zero,•,…
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
FT_X_trained = tfidf_text_train_df.join(scaled_train)

In [133]:
FT_X_trained.shape

(10165, 1967)

In [134]:
FT_X_trained.head()

,1,10,11,12,13,14,15,16,17,18,...,…,sentiment_score,abs_sent_score,punct_count,word_count,%adj,%verb,%adv,%noun,avg_word_length
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.378942,-0.584117,0.107380,1.787735,0.125383,0.155285,-0.298495,0.849950,-0.302474
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.069278,-1.026921,-0.712079,-0.409388,0.314773,0.438876,-0.540039,0.792049,-0.648888
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.069278,-1.026921,0.517110,-0.182099,-0.861440,0.552504,-0.080325,-0.273210,0.703670
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.069278,-1.026921,1.336569,0.045189,-0.277755,-0.448369,-0.540039,0.398204,0.460798
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.069278,-1.026921,-0.712079,-1.091253,-0.861440,0.830264,-0.540039,0.370299,-0.448944


In [ ]:
#TESTING DATA

In [135]:
text_test = X_test['text']
non_text_test = X_test.drop('text',axis = 1)

In [137]:
tfidf_text_test = tfidf_vector.transform(text_test)
tfidf_text_test_df = pd.DataFrame(tfidf_text_test.toarray())
tfidf_text_test_df.columns = list(tfidf_vector.get_feature_names())

In [141]:

scaled_test = scaler.transform(non_text_test)
scaled_test = pd.DataFrame(scaled_test)
scaled_test.columns = list(non_text_test.columns)

In [142]:
FT_X_test = tfidf_text_test_df.join(scaled_test)

In [144]:
FT_X_test.head()

,1,10,11,12,13,14,15,16,17,18,...,…,sentiment_score,abs_sent_score,punct_count,word_count,%adj,%verb,%adv,%noun,avg_word_length
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.535536,-0.162070,-0.302349,0.499767,0.524811,-0.284494,-0.200727,0.304024,0.439266
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.785376,-0.002937,0.107380,0.954344,-0.430162,-0.030790,0.726728,-0.371981,-0.120912
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.125591,0.681680,1.336569,1.711972,-0.118568,-0.210005,0.278208,-0.766206,-0.598902
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.753268,0.149277,-0.302349,-0.030573,-0.155712,-0.793998,-0.021816,0.792049,-0.006964
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.514004,-0.192858,-0.712079,-0.863965,-0.861440,-1.322372,-0.540039,0.130854,-1.236221


# Fit/Train Models

In [152]:
from sklearn.dummy import DummyClassifier

dummy_clf_1 = DummyClassifier(strategy='stratified')
dummy_clf_1.fit(FT_X_trained, y_train)

dummy_preds_1 = dummy_clf_1.predict(FT_X_test)

In [153]:
# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, dummy_preds_1))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, dummy_preds_1, digits=3))

[[2510 1000]
 [1043  454]]
                precision    recall  f1-score   support

non-propaganda      0.706     0.715     0.711      3510
    propaganda      0.312     0.303     0.308      1497

      accuracy                          0.592      5007
     macro avg      0.509     0.509     0.509      5007
  weighted avg      0.589     0.592     0.590      5007



In [154]:
from sklearn.dummy import DummyClassifier

dummy_clf_2 = DummyClassifier(strategy='most_frequent')
dummy_clf_2.fit(FT_X_trained, y_train)

dummy_preds_2 = dummy_clf_2.predict(FT_X_test)

In [155]:
# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, dummy_preds_2))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, dummy_preds_2, digits=3))

[[3510    0]
 [1497    0]]


/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

non-propaganda      0.701     1.000     0.824      3510
    propaganda      0.000     0.000     0.000      1497

      accuracy                          0.701      5007
     macro avg      0.351     0.500     0.412      5007
  weighted avg      0.491     0.701     0.578      5007



## Logistic Regression

In [147]:

# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression(C = 3, penalty = 'l2', solver='saga', class_weight = 'balanced')
# logreg_cv=GridSearchCV(logreg,grid,cv=10,scoring='f1_weighted')
clf_logreg = logreg.fit(FT_X_trained, y_train)


/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [148]:
logreg_test_preds = clf_logreg.predict(FT_X_test)

In [149]:
# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, logreg_test_preds))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, logreg_test_preds, digits=3))

[[2350 1160]
 [ 596  901]]
                precision    recall  f1-score   support

non-propaganda      0.798     0.670     0.728      3510
    propaganda      0.437     0.602     0.506      1497

      accuracy                          0.649      5007
     macro avg      0.617     0.636     0.617      5007
  weighted avg      0.690     0.649     0.662      5007



In [167]:

# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg_l1=LogisticRegression(C = 3, penalty = 'l1', solver='saga', class_weight = 'balanced')
# logreg_cv=GridSearchCV(logreg,grid,cv=10,scoring='f1_weighted')
clf_logreg_l1 = logreg.fit(FT_X_trained, y_train)
logreg_test_preds_l1 = clf_logreg_l1.predict(FT_X_test)

# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, logreg_test_preds_l1))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, logreg_test_preds_l1, digits=3))

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


[[2353 1157]
 [ 597  900]]
                precision    recall  f1-score   support

non-propaganda      0.798     0.670     0.728      3510
    propaganda      0.438     0.601     0.506      1497

      accuracy                          0.650      5007
     macro avg      0.618     0.636     0.617      5007
  weighted avg      0.690     0.650     0.662      5007



In [ ]:
# dummy_preds = dummy_clf.predict(FT_X_test)

## SVM

In [160]:
from sklearn import svm
clf_svc = svm.SVC(class_weight='balanced',C=3)
clf_svc.fit(FT_X_trained, y_train)


SVC(C=3, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [161]:
svc_preds = clf_svc.predict(FT_X_test)

In [162]:
# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, svc_preds))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, svc_preds, digits=3))

[[2590  920]
 [ 606  891]]
                precision    recall  f1-score   support

non-propaganda      0.810     0.738     0.772      3510
    propaganda      0.492     0.595     0.539      1497

      accuracy                          0.695      5007
     macro avg      0.651     0.667     0.656      5007
  weighted avg      0.715     0.695     0.703      5007



In [163]:
clf_svc_10 = svm.SVC(class_weight='balanced',C=10)
clf_svc_10.fit(FT_X_trained, y_train)
svc_preds_10 = clf_svc_10.predict(FT_X_test)

# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, svc_preds_10))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, svc_preds_10, digits=3))

[[2826  684]
 [ 781  716]]
                precision    recall  f1-score   support

non-propaganda      0.783     0.805     0.794      3510
    propaganda      0.511     0.478     0.494      1497

      accuracy                          0.707      5007
     macro avg      0.647     0.642     0.644      5007
  weighted avg      0.702     0.707     0.705      5007



In [165]:
clf_svc_2 = svm.SVC(class_weight='balanced',C=2)
clf_svc_2.fit(FT_X_trained, y_train)
svc_preds_2 = clf_svc_2.predict(FT_X_test)

# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, svc_preds_2))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, svc_preds_2, digits=3))

[[2490 1020]
 [ 579  918]]
                precision    recall  f1-score   support

non-propaganda      0.811     0.709     0.757      3510
    propaganda      0.474     0.613     0.534      1497

      accuracy                          0.681      5007
     macro avg      0.643     0.661     0.646      5007
  weighted avg      0.710     0.681     0.690      5007



In [166]:
clf_svc_01 = svm.SVC(class_weight='balanced',C=.1)
clf_svc_01.fit(FT_X_trained, y_train)
svc_preds_01 = clf_svc_01.predict(FT_X_test)

# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, svc_preds_01))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, svc_preds_01, digits=3))

[[2261 1249]
 [ 593  904]]
                precision    recall  f1-score   support

non-propaganda      0.792     0.644     0.711      3510
    propaganda      0.420     0.604     0.495      1497

      accuracy                          0.632      5007
     macro avg      0.606     0.624     0.603      5007
  weighted avg      0.681     0.632     0.646      5007



## Stochastic Gradient Descent

## SVD

In [169]:
from sklearn.decomposition import TruncatedSVD

In [219]:
# tfidf_vec = TfidfVectorizer(use_idf=True, norm='l2')
svd = TruncatedSVD(n_components=200, n_iter = 50)
# transformed_x_train = tfidf_vec.fit_transform(FT_X_trained)
# transformed_x_test = tfidf_vec.transform(FT_X_test)
# print('TF-IDF output shape:', transformed_x_train.shape)
x_train_svd = svd.fit_transform(FT_X_trained)
x_test_svd = svd.transform(FT_X_test)
print('LSA output shape:', x_train_svd.shape)
explained_variance = svd.explained_variance_ratio_.sum()
print("Sum of explained variance ratio: %d%%" % (int(explained_variance * 100)))

LSA output shape: (10165, 200)
Sum of explained variance ratio: 93%


In [224]:
# grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg_svd=LogisticRegression(C =3, penalty = 'l2', solver='lbfgs', class_weight = 'balanced', max_iter = 500)
# logreg_cv=GridSearchCV(logreg,grid,cv=10,scoring='f1_weighted')
clf_logreg_svd = logreg_svd.fit(x_train_svd, y_train)
logreg_test_preds_svd = clf_logreg_svd.predict(x_test_svd)

# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, logreg_test_preds_svd))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, logreg_test_preds_svd, digits=3))

[[2306 1204]
 [ 537  960]]
                precision    recall  f1-score   support

non-propaganda      0.811     0.657     0.726      3510
    propaganda      0.444     0.641     0.524      1497

      accuracy                          0.652      5007
     macro avg      0.627     0.649     0.625      5007
  weighted avg      0.701     0.652     0.666      5007



In [243]:
print(y_test[200:250], logreg_test_preds_svd[200:250])

2660     non-propaganda
14153    non-propaganda
11053    non-propaganda
10519    non-propaganda
10468    non-propaganda
5167     non-propaganda
3959     non-propaganda
11454    non-propaganda
11105        propaganda
1089     non-propaganda
9904     non-propaganda
9494     non-propaganda
2485     non-propaganda
7855     non-propaganda
9029     non-propaganda
13076    non-propaganda
2452     non-propaganda
1354     non-propaganda
1724         propaganda
6351         propaganda
2258     non-propaganda
9464         propaganda
13937    non-propaganda
3536         propaganda
8734         propaganda
9841         propaganda
14121        propaganda
10222    non-propaganda
3474     non-propaganda
9241     non-propaganda
11602        propaganda
412      non-propaganda
4909     non-propaganda
13718    non-propaganda
11461    non-propaganda
9679         propaganda
252      non-propaganda
14203        propaganda
6359     non-propaganda
7507         propaganda
12474    non-propaganda
7989     non-pro

In [242]:
df.loc[11797].text

'Then it’s up to us.'

In [177]:
from sklearn.linear_model import SGDClassifier

In [229]:
svd_sgd_clf = SGDClassifier(loss="log", penalty="l2", max_iter=500, class_weight = 'balanced')
svd_sgd_clf.fit(x_train_svd, y_train)
svd_sgd_predicts = svd_sgd_clf.predict(x_test_svd)

# Print the confusion matrix
print(sklearn.metrics.confusion_matrix(y_test, svd_sgd_predicts))

# Print the precision and recall, among other metrics
print(sklearn.metrics.classification_report(y_test, svd_sgd_predicts, digits=3))

[[2264 1246]
 [ 543  954]]
                precision    recall  f1-score   support

non-propaganda      0.807     0.645     0.717      3510
    propaganda      0.434     0.637     0.516      1497

      accuracy                          0.643      5007
     macro avg      0.620     0.641     0.616      5007
  weighted avg      0.695     0.643     0.657      5007

